# Part 3 Variants of Pooling

1) List three different kinds of pooling, and their corresponding module implemented in PyTorch.
The three types of pooling are:
- Max pooling: implemented in torch.nn.MaxPool1d/MaxPool2d/MaxPool3d
- Average pooling : implemented in torch.nn.AvgPool1d/AvgPool2d/AvgPool3d
- Power-average pooling: implemented in torch.nn.LPPool2d

2) Write down the mathematical forms of these three pooling modules.

Let C be the number of channels of a pixel, H and W be the height and width of the input image; The kernel size for pooling is (kH,kW); 

The 2d maximun pooling would return:
$$
Out(C_j,h,w) = Max_{m=0}^{kH-1} Max_{n=0}^{kW-1} In(C_j,stride[0]*h+m,stride[1]*w+n)
$$

The 2d average pooling would return:
$$
Out(C_j,h,w) = \frac{1}{kH*kW}\sum \limits_{m=0}^{kH-1} \sum \limits_{n=0}^{kW-1} In(C_j,stride[0]*h+m,stride[1]*w+n)
$$

The 2d Power average pooling would return:

$$
Out(C_j,h,w) = (\sum \limits_{m=0}^{kH-1} \sum \limits_{n=0}^{kW-1} (In(C_j,stride[0]*h+m,stride[1]*w+n))^p)^{\frac{1}{p}}
$$

Note that when $p = 1$ and $p = \infty$, power average pooling is equivalent to average pooling and maximun pooling respectively.


3) Pick one of the pooling listed and describe the reason for incorporating it into a deep
learning system.
For image classification we used max-pooling becauses it was experimentally shown to capture more translated invariant features.

References

http://yann.lecun.com/exdb/publis/pdf/boureau-icml-10.pdf

http://ieeexplore.ieee.org/stamp/stamp.jsp?arnumber=7172506

http://ais.uni-bonn.de/papers/icann2010_maxpool.pdf

# Part 4 t-SNE
### 1) What is the crowding problem and how does t-SNE alleviate it? Give details.

The crowding problem occurs when trying to map moderately distant points in high dimension space to 2 dimensions. The area of a sphere in a 2 dimensional space is smaller than the sphere with same radius in a high dimensional space, resulting in insufficient area to hold the same number of data points. In order to more accurately represent pairwise distances relations, the 2-dimensional map tends to assign higher distances to moderate distant points. 

The cost function of SNE is inversely related to the distance of the lower dimensional map between two points. As a result, the stochastic gradient descent process has a tendency to pulls clusters of data points towards their shared center. This phenomena is similar to the spring force and many other attraction forces in physics as mensioned in Maaten and Hinton's paper. Such attraction forces generally prevent cluster from being seperated.

t-SNE uses a t-distribution instead of gaussian distribution for $q_{j|i}$. With a stronger tail, t-distribution assigns a larger $q_{j|i}$ to the same pair of data points when the distance is large. Therefore the mapped data points in 2-dimensional space can be sufficiently represented by larger distances, allowing cluster gaps to form.

### 2) Please derive $\frac{\partial C}{\partial y_i}$ in detail.

### The following steps are also shown in Maaten and Hinton's Paper.

The definition of C is given by:
$$
C = \sum \limits_i \sum \limits_j p_{ij} log \frac{p_{ij}}{q_{ij}} = \sum \limits_i \sum \limits_j p_{ij}log(p_{ij})-\sum \limits_i \sum \limits_j p_{ij} log(q_{ij})
$$

Where:

$$
p_{ij} = \frac{exp(-\lvert \lvert x_i-x_j\rvert \rvert^2 /2 \sigma^2)}{\sum_{k \neq l}exp(-\lvert \lvert x_k-x_l\rvert \rvert^2 /2 \sigma^2)}
\\
q_{ij} = \frac{(1+\lvert \lvert y_i-y_j \rvert \rvert^2)^{-1}}{\sum_{k \neq l}(1+\lvert \lvert y_i-y_j \rvert \rvert^2)^{-1}}
$$


Let's define:
$$
d_{ij} = \lvert \lvert y_i-y_j \rvert \rvert \\
Z = \sum_{k \neq l}(1+d_{kl}^2)^{-1}
$$

We must have:
$$
(1+d_{ij}^2)^{-1} = q_{ij}Z
$$


Notice that in our definition of $C$, only $q_{ij}(d_{ij})$ and $q_{ji}(d_{ji})$ are functions of $y_i$. With symetry definition of SNE we have $d_{ij}=d_{ji}$. Therefore:

$$
\frac{\partial C}{\partial y_i} = 2 \sum_j \frac{\partial C}{\partial d_{ij}}(y_i-y_j)
$$

$\frac{\partial C}{\partial d_{ij}}$ is computed by:

\begin{split}
\frac{\partial C}{\partial d_{ij}} &= -p_{ij}\frac{\partial log(q_{ij}Z)}{d_{ij}}+\sum_{k\neq l}p_{kl}\frac{\partial log(Z)}{d_{ij}}\\
&=2\frac{p_{ij}}{q_{ij}Z}(1+d_{ij}^2)^{-2}-2\sum_{k\neq l}\frac{p_{kl}}{Z}(1+d_{ij}^2)^{-2}
\end{split}

Since $p_{kl}$ is a well-defined probability distribution, $\sum_{k\neq l}p_{kl}$ is 1.  Also by definition of Z we have $(1+d_{ij}^2)^{-1} = q_{ij}Z$ and $Z=\frac{(1+d_{ij}^2)^{-1}}{q_{ij}}$. Therefore:

\begin{split}
\frac{\partial C}{\partial d_{ij}} &=2p_{ij}(1+d_{ij}^2)^{-1}-2q_{ij}(1+d_{ij}^2)^{-1}
&=2(p_{ij}-q_{ij})(1+d_{ij}^2)^{-1}
\end{split}

Plug in the definition of $\frac{\partial C}{\partial d_{ij}}$ back, we get:
$$
\frac{\partial C}{\partial y_i} = 4(p_{ij}-q_{ij})(1+\lvert \lvert y_i-y_j \rvert \rvert ^2)^{-1}(y_i-y_j)
$$